# Day 15

## Part 1

In [51]:
def manhattan_distance(v1, v2):
    return int(abs(v1.real-v2.real) + abs(v1.imag-v2.imag))

In [52]:
def parse_line(line:str)->(complex,complex):
    '''
    Returns the locations for the Sensor and beacon as complex numbers
    
    '''
    s = line.split(':')[0].split('x=')[1]
    sx = int(s.split(',')[0])
    sy = int(s.split('y=')[1])
    b = line.split(':')[1].split('x=')[1]
    bx = int(b.split(',')[0])
    by = int(b.split('y=')[1].strip('\n'))
    return complex(sx,sy), complex(bx,by)

Just for kicks, let's find all the points that are covered by the current sensors

In [53]:
def get_closest_point_and_distance(s:complex, row:int)->(complex,int):
    return complex(s.real,row), manhattan_distance(s,complex(s.real,row))

In [54]:
def move_over_row(s:complex, cd:int, d:int)->list[complex]:
    ret_points = []
    this_distance = cd
    for i in range(1,d-cd+1):
        ret_points.append(complex(s.real-i,s.imag))
        ret_points.append(complex(s.real+i,s.imag))
    return ret_points

In [55]:
def get_points(s:complex, row:int, dist:int)->list[complex]:
    closest_point, closest_distance = get_closest_point_and_distance(s,row)
    if closest_distance > dist:
        return []
    ret_points = [closest_point]
    ret_points.extend(move_over_row(closest_point,closest_distance,dist))
    return ret_points

In [56]:
def get_answer_part_1(sensors:list[complex], beacons:list[complex], dists:list[int], row:int)->int:
    all_points = []
    count = 0
    for s,d in zip(sensors,dists):
        points = get_points(s,row,d)
        if beacons[count] in points: points.remove(beacons[count])
        all_points = list(set(all_points + points))
        count += 1
    return len(all_points)

In [57]:
with open('test_day15.txt') as input_text:
    sensors = []
    beacons = []
    dists = []
    for line in input_text:
        s,b = parse_line(line)
        sensors.append(s)
        beacons.append(b)
        dists.append(manhattan_distance(s,b))
    assert get_answer_part_1(sensors,beacons,dists,10)==26

In [58]:
with open('input_day15.txt') as input_text:
    sensors = []
    beacons = []
    dists = []
    for line in input_text:
        s,b = parse_line(line)
        sensors.append(s)
        beacons.append(b)
        dists.append(manhattan_distance(s,b))
    get_answer_part_1(sensors,beacons,dists,2000000)

## Part 2

In [59]:
def insert(r:list[int], recs:list[list[int]])->list[list[int]]:
    left = recs[0][0]
    index = 0
    flag = False
    while r[0] > left:
        index += 1
        try:
            left = recs[index][0]
        except IndexError:
            flag = True
            break
    if flag:
        recs.append(r)
    else:
        recs.insert(index,r)
    return recs

In [60]:
def merge_intervals(ranges: list[tuple[int, int]]):
    index = 0
    for i in range(1, len(ranges)):
        if ranges[index][1] >= ranges[i][0]:
            ranges[index][1] = max(ranges[index][1], ranges[i][1])
        else:
            index = index + 1
            ranges[index] = ranges[i]
    return ranges[0 : index + 1]

In [61]:
def compute_unique_point(sensors:list[list[complex]],dists:list[int], left:int,right:int)->complex:
    for row in range(left,right):
        recs = []
        for s,d in zip(sensors,dists):
            cp, cd = get_closest_point_and_distance(s, row)
            if cd <= d:
                rec = [max(left,cp.real-(d-cd)),min(cp.real+(d-cd),right)]
                # insert this rec into the current recs list to keep it sorted on the left index
                if recs != []:
                    recs = insert(rec,recs)
                else:
                    recs.append(rec)
        val = merge_intervals(recs)
        if len(val) > 1:
            # This is the row where the unique location exists
            return(complex(int(val[1][0] + val[0][1])//2, row))
    raise 'This should never be here'

In [62]:
with open('test_day15.txt') as input_text:
    sensors = []
    beacons = []
    dists = []
    for line in input_text:
        s,b = parse_line(line)
        sensors.append(s)
        beacons.append(b)
        dists.append(manhattan_distance(s,b))
    p = compute_unique_point(sensors,dists,0,21)
    assert int(p.real*4000000 + p.imag) == 56000011

In [63]:
with open('input_day15.txt') as input_text:
    sensors = []
    beacons = []
    dists = []
    for line in input_text:
        s,b = parse_line(line)
        sensors.append(s)
        beacons.append(b)
        dists.append(manhattan_distance(s,b))
    p = compute_unique_point(sensors,dists,0,4000000)
    print(int(p.real*4000000 + p.imag))

13071206703981
